# Better Way 33. 메타클래스로 서브클래스를 검증하자

메타클래스를 응용하는 간단한 예: 클래스를 올바르게 정의했는지 검증하는 것
- 보통 클래스 검증 코드는 클래스의 객체가 생성될 때 __init__ 메서드에서 실행됨 (BW 28에서 예제 참고)
- 메타클래스를 검증용으로 사용하면 오류를 더 빨리 일으킬 수 있음.

메타클래스가 표준 객체에는 어떻게 동작하는가
- 메타클래스는 type을 상속하여 정의함. 
- 기본으로 자체의 __new__ 메서드에서 연관된 class 문의 컨텐츠를 받음.
- 여기서 타입이 실제로 생성되기 전에 클래스 정보를 수정할 수 있음.
- 메타클래스는 클래스의 이름, 부모 ㅡㅋㄹ래스, class 본문에서 정의한 모든 클래스 속성에 접근 가능함.

In [1]:
class Meta(type):
    def __new__(meta, name, bases, class_dict):
        print((meta, name, bases, class_dict))
        return type.__new__(meta, name, bases, class_dict)
    
class MyClass(object, metaclass=Meta):
    stuff = 123
    
    def foo(self):
        pass

(<class '__main__.Meta'>, 'MyClass', (<class 'object'>,), {'foo': <function MyClass.foo at 0x7eff65fc2158>, '__module__': '__main__', '__qualname__': 'MyClass', 'stuff': 123})


클래스가 정의되기 전에 클래스의 모든 파라미터를 검증하려면 Meta.__new__ 메서드에 기능을 추가하면 된다.

In [2]:
class ValidatePolygon(type):
    def __new__(meta, name, bases, class_dict):
        # abstract class 검증하지 않음.
        if bases != (object,):
            if class_dict['sides'] < 3:
                raise ValueError('Polygons need 3+ sides')
        return type.__new__(meta, name, bases, class_dict)
    
class Polygon(object, metaclass=ValidatePolygon):
    sides = None
    
    @classmethod
    def interior_angles(cls):
        return (cls.sides - 2) * 180
    
class Triangle(Polygon):
    sides = 3

만약에 면이 3개 미만인 다각형을 정의하려고 하면 검증 코드가 class 문의 본문이 끝나자마자 class 문을 실패하게 만듦.
- 즉 이런 클래스를 정의하면 프로그램이 실행을 시작하지도 못함.

In [3]:
print('Before class')
class Line(Polygon):
    print('before sides')
    sides = 1
    print('After sides')
print('After class')

Before class
before sides
After sides


ValueError: Polygons need 3+ sides

핵심 정리
- 서브클래스 타입의 객체를 생성하기에 앞서 서브클래스가 정의 시점부터 제대로 구성되었음을 보장하려면 메타클래스를 사용하자.
- 파이썬2와 파이썬3의 메타클래스 문법은 약간 다르다. (파이썬2에서는 __metaclass__ 클래스 속성 사용)
- 메타클래스의 __new__ 메서드는 class 문의 본문 전체가 처리된 후에 실행된다.

# Better Way 34. 메타클래스로 클래스의 존재를 등록하자

메타클래스의 사용법 (2): 프로그램에 있는 타입을 자동으로 등록하는 것
- 등록(registration): 간단한 식별자(identifier)를 대응하는 클래스에 매핑하는 역방향 조회(reverse lookup) 수행 시 유용함.

In [4]:
import json

class Serializable(object):
    def __init__(self, *args):
        self.args = args
        
    def serialize(self):
        return json.dumps({'args':self.args})

In [5]:
class Point2D(Serializable):
    def __init__(self, x, y):
        super().__init__(x, y)
        self.x = x
        self.y = y
        
    def __repr__(self):
        return 'Point2D(%d, %d)' % (self.x, self.y)

In [6]:
point = Point2D(5, 3)
print('Object:', point)
print('Serialized:', point.serialize())

Object: Point2D(5, 3)
Serialized: {"args": [5, 3]}


In [8]:
class Deserializable(Serializable):
    @classmethod
    def deserialize(cls, json_data):
        params = json.loads(json_data)
        return cls(*params['args'])

In [9]:
class BetterPoint2D(Deserializable):
    def __init__(self, x, y):
        super().__init__(x, y)
        self.x = x
        self.y = y
        
    def __repr__(self):
        return 'BetterPoint2D(%d, %d)' % (self.x, self.y)

In [10]:
point = BetterPoint2D(5, 3)
print('Before: ', point)
data = point.serialize()
print('Serialized:', data)
after = BetterPoint2D.deserialize(data)
print('After:', after)

Before:  BetterPoint2D(5, 3)
Serialized: {"args": [5, 3]}
After: BetterPoint2D(5, 3)


이 방법의 문제점: 직렬화된 데이터에 대응하는 타입 (Point2D와 BetterPoint2D) 미리 알고 있을 경우에만 동작함.
- 이상적으로는 JSON으로 직렬화되는 클래스를 많이 갖추고
- 그중 어떤 클래스든 대응하는 파이썬 객체로 역직렬화하는 공통 함수를 하나만 두려고 할 것임.

In [26]:
class BetterSerializable(object):
    def __init__(self, *args):
        self.args = args
        
    def serialize(self):
        return json.dumps({
            'class': self.__class__.__name__, # 직렬화할 객체의 클래스 이름을 데이터에 포함시킨다.
            'args': self.args,
        })
    
    def __repr__(self):
        return '{}{}'.format(self.__class__.__name__, self.args)

In [27]:
registry = {} #클래스 이름을 해당 클래스의 객체 생성자에 매핑하고 이를 관리함.

def register_class(target_class):
    registry[target_class.__name__] = target_class
    
def deserialize(data): #어떤 클래스가 넘어와도 제대로 동작 가능함.
    params = json.loads(data)
    name = params['class']
    target_class = registry[name]
    return target_class(*params['args']) 

In [28]:
class EvenBetterPoint2D(BetterSerializable):
    def __init__(self, x, y):
        super().__init__(x, y)
        self.x = x
        self.y = y       

In [29]:
register_class(EvenBetterPoint2D) #deserialize가 항상 제대로 동작하려면, 잊지 않고 클래스를 등록해야 함.

In [30]:
point = EvenBetterPoint2D(5, 3)
print('Before: ', point)
data = point.serialize()
print('Serialized: ', data)
after = deserialize(data)
print('After: ', after)

Before:  EvenBetterPoint2D(5, 3)
Serialized:  {"args": [5, 3], "class": "EvenBetterPoint2D"}
After:  EvenBetterPoint2D(5, 3)


In [31]:
class Point3D(BetterSerializable):
    def __init__(self, x, y, z):
        super().__init__(x, y, z)
        self.x = x
        self.y = y
        self.z = z

In [32]:
point = Point3D(5, 9, -4)
data = point.serialize()
deserialize(data) #registry에 class name 등록하지 않았으므로 KeyError 발생함.

KeyError: 'Point3D'

- 문제점: BetterSerializable 상속하여 서브클래스 만들더라도, class 본문 이후에 register_class 호출하지 않으면 실제로 모든 기능을 사용하진 못함. 따라서 오류 발생 가능성이 높으며, 파이썬 3 클래스 데코레이터를 이용할 때도 누락이 발생할 수 있음. 
- 해결: 메타클래스를 이용하여, 서브클래스가 정의될 때 class 문을 가로채는 방법 (BW33 참고)으로 항상 register_class가 호출되도록 만들 수 있음. 메타클래스로 클래스 본문이 끝나자마자 새 타입을 등록하면 됨.

In [33]:
class Meta(type):
    def __new__(meta, name, bases, class_dict):
        cls = type.__new__(meta, name, bases, class_dict)
        register_class(cls)
        return cls
    
class RegisteredSerializable(BetterSerializable, metaclass=Meta):
    pass

In [34]:
class Vector3D(RegisteredSerializable):
    def __init__(self, x, y, z):
        super().__init__(x, y, z)
        self.x, self.y, self.z = x, y, z

In [35]:
v3 = Vector3D(10, -7, 3)
print('Before: ', v3)
data = v3.serialize()
print('Serialized: ', data)
print('After: ', deserialize(data))

Before:  Vector3D(10, -7, 3)
Serialized:  {"args": [10, -7, 3], "class": "Vector3D"}
After:  Vector3D(10, -7, 3)


메타클래스를 이용해 클래스를 등록하면 상속 트리가 올바르게 구축되어 있는 한 클래스 등록을 놓치지 않는다.
- 직렬화에 잘 동작하며,
- 데이터베이스 객체 관계 매핑,
- 플러그인 시스템,
- 시스템 후크에도 사용 가능함.

핵심 정리
- 클래스 등록은 모듈 방식의 파이썬 프로그램을 만들 때 유용한 패턴이다.
- 메타클래스를 이용하면 프로그램에서 기반 클래스로 서브클래스를 만들 때마다 자동으로 등록 코드를 실행할 수 있다.
- 메타클래스를 이용해 클래스를 등록하면 등록 호출을 절대 빠뜨리지 않으므로 오류를 방지할 수 있다.